# Toggle settings
Con questo semplice comando importiamo tutte le librerie necessarie e le funzioni che abbiamo scritto nel file settings.

In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Caricamento immagini
Visualizziamo prima immagine da tiles (2 stepsize)

In [2]:
# LOAD ALL IMAGES:
dt = 10 

dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt
# LOAD TRUE POSITIONS
df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)

In [3]:
img = cv2.imread(dict[str(idx+1)])
craters_det = detect(img)
craters_det_sort = sort_mat(craters_det)
TRI_det = find_triplets(craters_det_sort)

craters_cat = crater_catalogued([real_X[0],real_Y[0], real_Z[0]])
craters_cat_sort = sort_mat(craters_cat)
TRI_cat = find_triplets(craters_cat_sort)

In [250]:
triplet = picktrip(TRI_det, 0)
K_vet = compute_K_vet(triplet)
K_vet

{29.03349370010954, 40.442459851273014, 110.52404644861744}

In [249]:
KK 
for i in range(len(TRI_cat)):
    triplet = picktrip(TRI_cat, i)
    K_vet = compute_K_vet(triplet)
    KK.append(K_vet)

464

In [246]:

def compute_K_vet(triplet):
    a,b,c = compute_sides(triplet)
    A,B,C = findAngles(a,b,c)
    K_vet = set([A,B,C])
    if K_vet is not None:
        return K_vet


def compute_sides(triplet):
    a = np.linalg.norm(triplet[0][0:2]-triplet[1][0:2])
    b = np.linalg.norm(triplet[1][0:2]-triplet[2][0:2])
    c = np.linalg.norm(triplet[2][0:2]-triplet[0][0:2])
    return a,b,c



def find_triplet(craters_df_sort: np.array, idx: int):
    M1 = craters_df_sort.copy()
    pick1 = M1.iloc[idx]
    i = pick1.i
    j = pick1.j
    
    PICKS = M1[ (M1.j == j-1) | (M1.j == j+1) | (M1.i == i+1) | (M1.i == i - 1) \
                | (M1.j == j-2) | (M1.j == j+2) | (M1.i == i+2) | (M1.i == i - 2)]
    ind = 0
    HP = np.zeros(2)
    for pick in PICKS.iloc:
        pick2 = pick
        dist_12 = np.linalg.norm(pick2[0:2]-pick1[0:2])
        if dist_12 > pick2.R + pick1.R:
            hp = np.hstack([ind, dist_12])  
            HP = np.vstack([HP, hp])
        ind +=1
    HP = HP[1:,:].copy() # Remove first zeros
    HP.view('f8,f8').sort(order=['f1'], axis=0) # Order by Eu Distance
    crater1 = pick1
    crater2 = PICKS.iloc[int(HP[0,0])]
    crater3 = PICKS.iloc[int(HP[1,0])]
    return [crater1, crater2, crater3]



def find_triplets(df):
    N, triplet_list = len(df), []
    for idx in range(N):
        # 1st:
        if idx == 0:
            c1,c2,c3 = find_triplet(df, 0)
            if np.all([c1,c2,c3]) != None: # If they exist
                entry = set([c1.name, c2.name, c3.name])
                triplet_list.append(entry)
        # 2nd to end:
        else:
            c1,c2,c3 = find_triplet(df, idx)
            if np.all([c1,c2,c3]) != None: # If they exist
                entry = set([c1.name, c2.name, c3.name]) # Note: set() make order meaningless
                if entry not in triplet_list:
                    triplet_list.append(entry)
        printProgressBar(idx, N, prefix="Progress:", suffix="Complete", length=50)

    return triplet_list

def picktrip(TRI, idx):
    triplet, IDs = TRI[idx], []
    
    for index in triplet: IDs.append(index)
    
    crat1 = craters_det_sort.iloc[IDs[0]]
    crat2 = craters_det_sort.iloc[IDs[1]]
    crat3 = craters_det_sort.iloc[IDs[2]]
    return crat1, crat2,crat3



def findAngles( a, b, c): 
    # applied cosine rule 
    A = np.arccos((b * b + c * c - a * a) / 
                              (2 * b * c)) 
    B = np.arccos((a * a + c * c - b * b) / 
                              (2 * a * c)) 
    C = np.arccos((b * b + a * a - c * c) / 
                              (2 * b * a)) 
    # convert into degrees 
    A, B, C = np.rad2deg(A),np.rad2deg(B),np.rad2deg(C)
    return A,B,C 

In [217]:
df = pd.read_cs

# Matching: 

In [7]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [8]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [9]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [10]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [11]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)